''' https://dummyjson.com/users'''

In [0]:
%fs ls

In [0]:
dbutils.fs.ls('/Volumes/workspace/default/tmp_customer/customer_transactions.csv')

In [0]:
order_df = spark.read.csv('/Volumes/workspace/default/tmp_customer/customer_transactions.csv', header=True)

In [0]:
display(order_df)



In [0]:
df.printSchema()
df.count()
df.show()
df.show(5)

In [0]:
user_list = list(df.select('user_id').toPandas()['user_id'])
print(user_list)

In [0]:
import requests 

base_url = 'https://dummyjson.com/users'

def customer_data_fetch():
    customer_api_response = []
    for user in user_list:
        response = requests.get(f"{base_url}/{user}")
        if response.status_code == 200:
            customer_api_response.append(response.json())
        else:
            print(f"Error: {response.status_code}")
    return customer_api_response

       

In [0]:
customer_api_response = customer_data_fetch()

print(customer_api_response)


In [0]:
requests.get("https://dummyjson.com/users").json()

In [0]:

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import *
from pyspark.sql.window import *

# Create a Spark session
spark = (
    SparkSession.builder 
    .appName("thebigdatashow.me")
    .getOrCreate()
)

In [0]:
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("firstName", StringType(), True),
                                            StructField("lastName", StringType(), True),
                                            StructField("email", StringType(), True),
                                            StructField("phone" , StringType(), True),
                                            StructField("address", StringType(), True)
                                            ])

In [0]:
import json

dbutils.fs.put(
    "/Volumes/workspace/default/tmp_customer/users.json",
    json.dumps(customer_api_response),
    overwrite=True
)

df = spark.read.json("/Volumes/workspace/default/tmp_customer/users.json" , schema=schema)
df.show()


In [0]:
display(df)

In [0]:
add_schema = StructType([
    StructField('address', StringType(),True),
    StructField('city', StringType(),True),
    StructField('state', StringType(),True),
    StructField('stateCode', StringType(),True),
    StructField('postalCode', StringType(),True),
    StructField('coordinates', StructType([
         StructField('lat', StringType(),True),
         StructField('lng', StringType(),True)
     ]),True),
     StructField('country', StringType(),True)
])

In [0]:
df.show(1, truncate=False)

In [0]:
df.printSchema()

In [0]:
new_df = (
        df
          .withColumnRenamed('address', "address_parent")
          .withColumn("jsonData" , from_json(col("address_parent"), add_schema))
            .select("*" , "jsonData.*")
            .drop("address_parent","jsonData"))

new_df.show(truncate=False)


In [0]:
display(new_df)

In [0]:
final_df = order_df.join(new_df, order_df.user_id == new_df.id, "left")

display(final_df)